In [ ]:
import numpy as np
%pylab
%matplotlib inline
%load_ext autoreload
%autoreload 2

from Functions import image_processing as ip
from Functions import seed_detection as sd
from Functions import seeded_region_growing as srg
from Functions import old_seeded_region_growing as old_srg
import skimage.io as sk
from Functions import region_merging as rm
from Functions import dice_score as ds
from Functions import segmentation as seg

7 Evaluation measures
<br>
7.1 Dice score
<br>
The dice score is a method to determine the accuracy of the segmentation. Thereby, a ground truth image is compared with
the result image of the algorithm. The ground truth image is a given result picture. The dice score is defined as follows:
<br>
$$ DSC = \frac{ 2 \times |S \cap G|}{|S| + |G|} $$
<br>
The intersection is the number of pixels that have the same value in the ground truth and result image. The doubled
intersection is divided by the total number of pixels in the ground truth and result image. This delivers a number
between zero and one. The higher the value the better was the performance of the algorithm.
<br>
Another implementation of the dice score would be weighting equally between background and cell pixels. This makes it possible
to consider even small regions, for example when there are proportionally few cells compared to the background.

$$\overline{DSC}=\frac{1}{2} \sum\limits_{i=1}^{2} \left(\frac{2 \times |S_i \cap G_i|}{|S_i|+|G_i|}\right)$$

Thereby, two regions were considered: cells and background. For each region, the dice score was calculated individually
and then weighted equally. With that every region has the same influence on the result in spite of the size.
The background was defined as the largest region and received another value than every other region in order to create a
binary array. With that the ground truth and the result image are comparable and do not depend on different region numbers for every cell.
The weighted dice score is important because the background has often a higher proportion in the total image. So loosing cells,
could not have such a high impact on the unweighted dice score as desired.

equation for standard deviation
$$ \sigma = \sqrt{\left(\frac{1}{9}\sum\limits_{i=1}^{9}(x_i-\overline{x}\right)^2}$$
<br>
$$x_i$$
$$\overline{x}$$

equation similarity
$$s=1-\frac{\sigma}{\sigma_{max}}$$
<br>
$$\sigma$$
$$\sigma_{max}$$


equation euclidean distance
$$d_{max} = \mathop{max}_{1\leq i\leq 8} \left\{ d_i = \frac{x-x_i}{x} \right\}$$
<br>
$$d_{max}$$
$$d_i$$
$$x$$
$$x_i$$


To test, which preprocessing method leads to the best result, different clipping methods and filters were compared. The
total algorithm for seeded region growing was applied to a fraction of an image. Using the weighted and unweighted dice
score, the best parameters where defined for every image. Thereby the focus was laid on the optimization of the weighted
dice score. The following shows the different preprocessed images:

In [ ]:
image_intensity = sk.imread("Data/N2DH-GOWT1/img/t31.tif")
image_intensity_small = image_intensity[400:500, 300:400]
image_clipped = ip.image_clipping(image_intensity_small,0.1*np.amax(image_intensity_small),0.9*np.amax(image_intensity_small))
image_clipped_extreme = ip.image_clipping_extreme(image_intensity_small, 0.03*np.amax(image_intensity_small), 0.1*np.amax(image_intensity_small))
image_median = ip.median_filter(image_intensity_small, 5)
image_gauss = ip.gaussian_filter(image_intensity_small, 3)
image_anisotropic = ip.anisotropic_filter(image_intensity_small)
ip.show_three_images_colorbar(image_intensity_small, image_clipped, image_clipped_extreme, 0.45)
ip.show_three_images_colorbar(image_median, image_gauss, image_anisotropic, 0.45)

On these pictures the seeded region growing algorithm was conducted the parameters were also determined individually for every preprocessed image.

In [ ]:
image_gt = sk.imread("Data/N2DH-GOWT1/gt/man_seg31.tif")
image_gt_small = image_gt[400:500, 300:400]
image_small_segmented = seg.seeded_segmentation(image_intensity_small, image_gt_small, 0.9, 0.1, 150)
image_clipped_segmented = seg.seeded_segmentation(image_clipped, image_gt_small, 0.01, 0.1, 200)
image_clipped_extreme_segmented = seg.seeded_segmentation(image_clipped_extreme, image_gt_small, 0.9, 0.1, 200)
image_median_segmented = seg.seeded_segmentation(image_median, image_gt_small, 0.9, 0.1, 150)
image_gauss_segmented = seg.seeded_segmentation(image_gauss, image_gt_small, 0.9, 0.1, 200)
image_anisotropic_segmented = seg.seeded_segmentation(image_anisotropic, image_gt_small, 0.01, 0.2, 200)
ip.show_three_images_colorbar(image_small_segmented, image_clipped_segmented, image_clipped_extreme_segmented, 0.45)
ip.show_three_images_colorbar(image_median_segmented,image_gauss_segmented, image_anisotropic_segmented, 0.45)

It is visible that the algorithm is working well on every image. The unprocessed image has already a dice score of 0.988.
The highest possible dice score is 1 so preprocessing cannot increase the result significantly.
Only the median filter increased the result with a weighted dice score of 0.989 slightly. All other filters and clipping
methods decreased the dice score a little. The Gauss filter performed the worst on this data set and decreased the dice score
to 0.94. This might be because, the Gauss filter blurs edges and thereby makes the identification of cells harder.
Otherwise, the range between all values of the dice score is not significantly different, the performance on other, more complicated
datasets would be interesting.

